<a href="https://colab.research.google.com/github/JoHussien/FacialExpressionsDetector/blob/main/GPUComputingAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1awgxw55
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1awgxw55
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


#Question 1
Write a program to perform matrix multiplication of any two floating-point matrices A and B. A and B do not need
to be square matrices; however, their dimensions should be compatible. i.e, the number of columns of A has to be
equal to the number of rows of B. A and B should be initialized to random values by the host, but their dimensions
should be specified in the program using #define directives.

## Matrix Multiplication --> Sequential Implementation


In [ ]:
%%cu
#include<cuda.h>
#include<stdio.h>
#include <cstdlib>
#include<iostream>
#include <time.h>



#define  Height1 2000
#define  Width1  2000
#define  Height2 2000
#define  Width2  2000
using namespace std;

void MatrixMulSequential(float* M, float* N,float* P)
  {
    

    for(int i = 0; i < Height1; ++i)
        for(int j = 0; j < Width2; ++j)
            {   
                float Pvalue = 0;
                for(int k = 0; k < Width1; ++k)
                  Pvalue += M[i * Width1 + k] * N[k * Width2+ j];
                P[i*Width2+j]=Pvalue;
            }
  
  }

void intialize_matrix(float *M,int ROWS,int COLS){
    for(int i=0; i<ROWS;i++){
          for(int j=0;j<COLS;j++){
            //M[j+i*COLS] =rand()%256;
             M[j+i*COLS] =j+i*COLS;
      }
  }
}
void Print_matrix(float *M,int ROWS,int COLS){
    for(int i=0; i<ROWS;i++){
          for(int j=0;j<COLS;j++){
            cout<<M[j+i*COLS]<<" ";
      }
      cout<<endl;
  }   
}
bool validation(){

  if (Width1!=Height2){
      cout<<"There is an error in the dims of the matrices"<<endl; 
      return 0;
  }
  return 1;
  
    
}


int main(){
    srand(time(0));
    
    float *M,*N,*Pout;
    
    
    if (!validation())
        return 0;
    

    M=(float*) malloc(Width1*Height1*sizeof(float));
    N=(float*) malloc(Width2*Height2*sizeof(float));
    Pout=(float*) malloc(Width2*Height1*sizeof(float));
    cout<<"Size of M is: "<<Height1<<"x"<<Width1<<endl;
    cout<<"Size of N is: "<<Height2<<"x"<<Width2<<endl;

    intialize_matrix(M,Height1,Width1);
    intialize_matrix(N,Height2,Width2);
    //cout<<"M"<<endl;
    //Print_matrix(M,Height1,Width1);
    //cout<<endl;
    //cout<<"N"<<endl;
    //Print_matrix(N,Height2,Width2);

    cout<<endl;
    clock_t start = clock();
    MatrixMulSequential(M,N,Pout);
    clock_t stop = clock();
    double time_spent = (double)(stop - start) / CLOCKS_PER_SEC;
    cout<<"Time taken to complete the computation is: "<<time_spent<<" seconds.\n";
    //cout<<"P"<<endl;
    // Print_matrix(Pout,Height1,Width2);
  //  cout<<Pout[0]<<" "<<Pout[1];
     //cout<<Pout[1999]<<" "<<Pout[1999];
    //cout<<"Finished Computing P matrix"<<endl;
    return 0;
}


Size of M is: 2000x2000
Size of N is: 2000x2000

Time taken to complete the computation is: 65.8551 seconds.



## Matrix Multiplication --> Parallel Implementation

Your parallel implementation should use multiple
2D blocks of threads but no shared memory. You program should print the performance of the sequential and
parallel versions in GFLOPS. Provide results for each of the following block sizes: 16x16, and 32x32 (Teams of 3 should also report on block sizes of 4x4 and 8x8). What is the GPU utilization in each case? Note: for testing
purposes, you try relatively small values for the dimensions of the matrices, but to provide timing results, A and B
matrices have to be large enough to make parallelism useful (try values larger than 1000 for each dimension).

In [ ]:
!nvidia-smi

Mon Nov  7 23:35:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!nvidia-smi


%%cu
#include<cuda.h>
#include<stdio.h>
#include <cstdlib>
#include<iostream>
#include <time.h>



#define  Height1 2000
#define  Width1  2000
#define  Height2 2000
#define  Width2  2000
using namespace std;

__global__
void MatrixMulKernel(float* M, float* N,float* P)
  {
    
    int Row = blockIdx.y*blockDim.y + threadIdx.y;
    int Col = blockIdx.x*blockDim.x + threadIdx.x;
    
    float Pvalue = 0;
    if (Row<Height1 && Col<Width2)
      {
        for (int k = 0; k < Width1; ++k){   
          Pvalue += M[Row*Width1+k] * N[k*Width2+Col];
        }
        
        P[Row*Width2+Col] = Pvalue;
      }
  }
void MatrixMulOnDevice(float* M, float* N, float* P)

  {
  int size1 = Width1 * Height1 * sizeof(float);
  int size2 = Width2 * Width1 * sizeof(float);
  int size3 = Height1 * Width2 * sizeof(float);
  
  float *d_M, *d_N, *d_P;
  cudaMalloc((void **)&d_M, size1);
   cudaError_t err = cudaMalloc((void **) &d_M, size1);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaMemcpy(d_M, M, size1, cudaMemcpyHostToDevice);
     err = cudaMemcpy(d_M, M, size1, cudaMemcpyHostToDevice);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaMalloc((void **)&d_N, size2);
   err = cudaMalloc((void **) &d_N, size2);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaMemcpy(d_N, N, size2, cudaMemcpyHostToDevice);
  err =  cudaMemcpy(d_N, N, size2, cudaMemcpyHostToDevice);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaMalloc((void **)&d_P, size3);
    err = cudaMalloc((void **) &d_P, size3);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  
  dim3 dimGrid(ceil(Width2/32.0), ceil(Height1/32.0), 1);

  dim3 dimBlock(32,32, 1);
  clock_t start = clock();
  MatrixMulKernel<<<dimGrid, dimBlock>>>(d_M, d_N, d_P);
  cudaDeviceSynchronize();
  err= cudaDeviceSynchronize();
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  clock_t stop = clock();
  double time_spent = (double)(stop - start) / CLOCKS_PER_SEC;
  cout<<"Time taken to complete only the Kernel computation is: "<<time_spent<<" seconds.\n";
  
  cudaMemcpy(P, d_P, size3, cudaMemcpyDeviceToHost);
     err=  cudaMemcpy(P, d_P, size3, cudaMemcpyDeviceToHost);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaFree(d_M); 
  cudaFree(d_N); cudaFree (d_P);
  }

void intialize_matrix(float *M,int ROWS,int COLS){
    for(int i=0; i<ROWS;i++){
          for(int j=0;j<COLS;j++){
            //M[j+i*COLS] =rand()%256;
             M[j+i*COLS] =j+i*COLS;
      }
  }
}
void Print_matrix(float *M,int ROWS,int COLS){
    for(int i=0; i<ROWS;i++){
          for(int j=0;j<COLS;j++){
            cout<<M[j+i*COLS]<<" ";
      }
      cout<<endl;
  }   
}
bool validation(){

  if (Width1!=Height2){
      cout<<"There is an error in the dims of the matrices"<<endl; 
      return 0;
  }
  return 1;
  
    
}


int main(){
    srand(time(0));
    
    float *M,*N,*Pout;
    
    
    if (!validation())
        return 0;
    
    cout<<"Size of M is: "<<Height1<<"x"<<Width1<<endl;
    cout<<"Size of N is: "<<Height2<<"x"<<Width2<<endl;

    M=(float*) malloc(Width1*Height1*sizeof(float));
    N=(float*) malloc(Width2*Height2*sizeof(float));
    Pout=(float*) malloc(Width2*Height1*sizeof(float));

    intialize_matrix(M,Height1,Width1);
    intialize_matrix(N,Height2,Width2);
    cout<<"M"<<endl;
  //  Print_matrix(M,Height1,Width1);
    cout<<endl;
    cout<<"N"<<endl;
  //  Print_matrix(N,Height2,Width2);

    cout<<endl;
    clock_t start = clock();
    MatrixMulOnDevice(M,N,Pout);
    cudaDeviceSynchronize();
    cudaError_t err1= cudaDeviceSynchronize();
   if (err1!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err1), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
    clock_t stop = clock();
    double time_spent = (double)(stop - start) / CLOCKS_PER_SEC;
    cout<<"Time taken to complete the entire wrapper computation is: "<<time_spent<<" seconds.\n";

    cout<<"Finished Computing P matrix"<<endl;

    //cout<<"P"<<endl;
 // Print_matrix(Pout,Height1,Width2);
  cout<<Pout[0]<<" "<<Pout[1];
    return 0;
}


Size of M is: 2000x2000
Size of N is: 2000x2000
M

N

Time taken to complete only the Kernel computation is: 0.048754 seconds.
Time taken to complete the entire wrapper computation is: 0.301577 seconds.
Finished Computing P matrix
5.32934e+12 5.32934e+12


###Speedup for 1000x500 * 500x1000 matrices
Speedup = Time in Sequential / Time in Parallel

speedup of kernel only = 1.77262/0.0015913 = 111x times speedup

speedup of whole wrapper function = 1.77262/0.308345 = 6x times speedup

###Speedup for 1000x1000 matrices
Speedup = Time in Sequential / Time in Parallel

speedup of kernel only = 3.54652/0.031297 = 113x times speedup

speedup of whole wrapper function = 3.54652/0.323412 = 11x times speedup

## Question 2 Tiling and Shared Memory

Repeat Task 1 using shared memory (and tiling) for the parallel implementation. 


In [ ]:
%%cu
#include<cuda.h>
#include<stdio.h>
#include <cstdlib>
#include<iostream>
#include <time.h>



#define  Height1 2000
#define  Width1  2000
#define  Height2 2000
#define  Width2  2000
#define TILE_WIDTH 16
using namespace std;

__global__
void MatrixMulKernel(float* M, float* N,float* P)
  {
    
        
    
    
    __shared__ float Mds[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Nds[TILE_WIDTH][TILE_WIDTH];
   
    int bx=blockIdx.x;int by=blockIdx.y;
    int tx=threadIdx.x;int ty=threadIdx.y; 
    int Row = by*TILE_WIDTH + ty;
    int Col = bx*TILE_WIDTH + tx;
    
    float Pvalue = 0;
    for(int ph=0;ph < ceil(Width1/(float)TILE_WIDTH);++ph)
      {      
        if(Row<Height1 && (ph*TILE_WIDTH+tx)<Width1)
          Mds[ty][tx]=M[Row*Width1+ph*TILE_WIDTH+tx];
    
       if((ph*TILE_WIDTH+ty)<Height2 && Col< Width2)
          Nds[ty][tx]=N[(ph*TILE_WIDTH+ty)*Width2+Col];
      
        __syncthreads();
     
        for (int k = 0; k < TILE_WIDTH; ++k){  
            Pvalue += Mds[ty][k] * Nds[k][tx]; 
          }
       
       __syncthreads();
      }
 
   if (Row<Height1 && Col<Width2){
      P[Row*Width2+Col] = Pvalue;
   }
   
   

    
  }
   
void MatrixMulOnDevice(float* M, float* N, float* P)

  {
  int size1 = Width1 * Height1 * sizeof(float);
  int size2 = Width2 * Width1 * sizeof(float);
  int size3 = Height1 * Width2 * sizeof(float);
  
  float *d_M, *d_N, *d_P;
  cudaMalloc((void **)&d_M, size1);
  cudaError_t err = cudaMalloc((void **) &d_M, size1);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaMemcpy(d_M, M, size1, cudaMemcpyHostToDevice);
   err = cudaMemcpy(d_M, M, size1, cudaMemcpyHostToDevice);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaMalloc((void **)&d_N, size2);
   err = cudaMalloc((void **)&d_N, size2);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaMemcpy(d_N, N, size2, cudaMemcpyHostToDevice);
    err = cudaMemcpy(d_N, N, size2, cudaMemcpyHostToDevice);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
   
  cudaMalloc((void **)&d_P, size3);
     err = cudaMalloc((void **)&d_P, size3);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }

  dim3 dimGrid(ceil(Width2/(float)TILE_WIDTH), ceil(Height1/(float)TILE_WIDTH), 1);
  dim3 dimBlock(TILE_WIDTH,TILE_WIDTH, 1);
  
  clock_t start = clock();
  MatrixMulKernel<<<dimGrid, dimBlock>>>(d_M, d_N, d_P);
  cudaDeviceSynchronize();
       err =  cudaDeviceSynchronize();
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  clock_t stop = clock();
  double time_spent = (double)(stop - start) / CLOCKS_PER_SEC;
  cout<<"Time taken to complete only the Kernel computation is: "<<time_spent<<" seconds.\n";
    
  cudaMemcpy(P, d_P, size3, cudaMemcpyDeviceToHost);
        err = cudaMemcpy(P, d_P, size3, cudaMemcpyDeviceToHost);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
   
  cudaFree(d_M); cudaFree(d_N); cudaFree (d_P);
  }

void intialize_matrix(float *M,int ROWS,int COLS){
    for(int i=0; i<ROWS;i++){
          for(int j=0;j<COLS;j++){
            //M[j+i*COLS] =rand()%256;
             M[j+i*COLS] =j+i*COLS;
 

      }
  }
}
void Print_matrix(float *M,int ROWS,int COLS){
    for(int i=0; i<ROWS;i++){
          for(int j=0;j<COLS;j++){
            cout<<M[j+i*COLS]<<" ";
      }
      cout<<endl;
  }   
}
bool validation(){

  if (Width1!=Height2){
      cout<<"There is an error in the dims of the matrices"<<endl; 
      return 0;
  }
  return 1;
  
    
}


int main(){
    srand(time(0));
    
    float *M,*N,*Pout;
    
    
    if (!validation())
        return 0;
        
    cout<<"Size of M is: "<<Height1<<"x"<<Width1<<endl;
    cout<<"Size of N is: "<<Height2<<"x"<<Width2<<endl;

    M=(float*) malloc(Width1*Height1*sizeof(float));
    N=(float*) malloc(Width2*Height2*sizeof(float));
    Pout=(float*) malloc(Width2*Height1*sizeof(float));

    intialize_matrix(M,Height1,Width1);
    intialize_matrix(N,Height2,Width2);
    //cout<<"M"<<endl;
    //Print_matrix(M,Height1,Width1);
    cout<<endl;
    //cout<<"N"<<endl;
    //Print_matrix(N,Height2,Width2);

    cout<<endl;
    clock_t start = clock();
    MatrixMulOnDevice(M,N,Pout);
    cudaDeviceSynchronize();
       cudaError_t err1 =   cudaDeviceSynchronize();
   if (err1!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err1), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
    clock_t stop = clock();
    double time_spent = (double)(stop - start) / CLOCKS_PER_SEC;
    cout<<"Time taken to complete the entire wrapper computation is: "<<time_spent<<" seconds.\n";

    cout<<"Finished Computing P matrix"<<endl;
//    cout<<"P"<<endl;
  // Print_matrix(Pout,Height1,Width2);


 cout<<Pout[0]<<" "<<Pout[1];
    return 0;
}

Size of M is: 2000x2000
Size of N is: 2000x2000


Time taken to complete only the Kernel computation is: 0.047152 seconds.
Time taken to complete the entire wrapper computation is: 0.29152 seconds.
Finished Computing P matrix
5.32934e+12 5.32934e+12


###Speedup for 1000x500 * 500x1000 matrices
Speedup = Time in Sequential / Time in Parallel

speedup of kernel only = 1.77262/0.003176 = 558x times speedup

speedup of whole wrapper function = 1.77262/0.281566 = 6x times speedup

###Speedup for 1000x1000 matrices
Speedup = Time in Sequential / Time in Parallel

speedup of kernel only = 3.54652/0.00624 = 568x times speedup

speedup of whole wrapper function = 3.54652/0.292596 = 12x times speedup

## Question 3 --> Different Thread Granularities (Done coding 💃 💃)

Repeat Task 2 with thread granularities of 2 and 4. i.e, each thread block will compute 2 or 4 elements in the
output matrix (Each block of threads will be responsible for computing 2 or 4 output tiles). Teams of 3 should also
report on thread granularity of 8.



In [ ]:
 %%cu
#include<cuda.h>
#include<stdio.h>
#include <cstdlib>
#include<iostream>
#include <time.h>



#define  Height1 2000
#define  Width1  2000
#define  Height2 2000
#define  Width2  2000
#define TILE_WIDTH 16
#define granularity 8
using namespace std;

__global__
void MatrixMulKernel(float* M, float* N,float* P)
  {
    
        
    
      
    __shared__ float Mds[TILE_WIDTH][TILE_WIDTH*granularity];
    __shared__ float Nds[TILE_WIDTH][TILE_WIDTH*granularity];
   
    int bx=blockIdx.x;
   
    int by=blockIdx.y;
    int tx=threadIdx.x;
    int ty=threadIdx.y; 
    
    int Row = by*TILE_WIDTH + ty;
    int Col = bx*TILE_WIDTH + tx;

    int Col1=Col*granularity;
    int tx1=tx*granularity;
    

    
    float Pvalue[granularity];
   for (int i=0;i<granularity;i++){
      Pvalue[i]=0;
 }

    for(int ph=0;ph < ceil(Width1/((float)TILE_WIDTH));++ph)
      {     
          if((Row)<Height1 && (ph*TILE_WIDTH+tx)<Width1)
          {
                        Mds[ty][tx]=M[(Row)*Width1+ph*TILE_WIDTH+tx];
                        
          }
          for(int j =0;j<granularity;j++)
            {
                      
                      if((ph*TILE_WIDTH+ty)<Height2 && (Col1+j)< Width2){
                        Nds[ty][tx1+j]=N[(ph*TILE_WIDTH+ty)*Width2+(Col1+j)];
                      
                    }
                      
                    
                  __syncthreads();
            
                      for (int k = 0; k < TILE_WIDTH; ++k){  
                          Pvalue[j] += Mds[ty][k] * Nds[k][tx1+j];
                         
                        }
               
     
       __syncthreads();
            }
      }
    for(int j =0;j<granularity;j++)
        {
          if (Row<Height1 && Col1+j<Width2){
              //if(j==0) //in order not repeat the same threads
              //printf("tx:%d,ty:%d\n",tx,ty); //to check how many thread working
            P[Row*Width2+Col1+j] = Pvalue[j];
          }
    
        }
      

   
    
  }
   
void MatrixMulOnDevice(float* M, float* N, float* P)

  {
  int size1 = Width1 * Height1 * sizeof(float);
  int size2 = Width2 * Width1 * sizeof(float);
  int size3 = Height1 * Width2 * sizeof(float);
  
  float *d_M, *d_N, *d_P;
  cudaMalloc((void **)&d_M, size1);
  cudaError_t err = cudaMalloc((void **)&d_M, size1);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaMemcpy(d_M, M, size1, cudaMemcpyHostToDevice);
        err = cudaMemcpy(d_M, M, size1, cudaMemcpyHostToDevice);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaMalloc((void **)&d_N, size2);
          err =   cudaMalloc((void **)&d_N, size2);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaMemcpy(d_N, N, size2, cudaMemcpyHostToDevice);
          err = cudaMemcpy(d_N, N, size2, cudaMemcpyHostToDevice);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaMalloc((void **)&d_P, size3);
          err = cudaMalloc((void **)&d_P, size3);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  
  

  dim3 dimGrid(ceil((Width2)/((float)TILE_WIDTH)), ceil(Height1/(float)TILE_WIDTH), 1);
  dim3 dimBlock(ceil(TILE_WIDTH),TILE_WIDTH, 1);
  clock_t start = clock();
  MatrixMulKernel<<<dimGrid, dimBlock>>>(d_M, d_N, d_P);
  cudaDeviceSynchronize();
  err = cudaDeviceSynchronize();
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  clock_t stop = clock();
  double time_spent = (double)(stop - start) / CLOCKS_PER_SEC;
  cout<<"Time taken to complete only the Kernel computation is: "<<time_spent<<" seconds.\n";
  cudaMemcpy(P, d_P, size3, cudaMemcpyDeviceToHost);
  err =cudaMemcpy(P, d_P, size3, cudaMemcpyDeviceToHost);
   if (err!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
  cudaFree(d_M); cudaFree(d_N); cudaFree (d_P);
  }

void intialize_matrix(float *M,int ROWS,int COLS){
    for(int i=0; i<ROWS;i++){
          for(int j=0;j<COLS;j++){
            //M[j+i*COLS] =rand()%256;
             M[j+i*COLS] =j+i*COLS;
      }
  }
}
void Print_matrix(float *M,int ROWS,int COLS){
    for(int i=0; i<ROWS;i++){
          for(int j=0;j<COLS;j++){
            cout<<M[j+i*COLS]<<" ";
      }
      cout<<endl;
  }   
}
bool validation(){

  if (Width1!=Height2){
      cout<<"There is an error in the dims of the matrices"<<endl; 
      return 0;
  }
  return 1;
  
    
}


int main(){
    srand(time(0));
    
    float *M,*N,*Pout;
    
    
    if (!validation())
        return 0;
    

    M=(float*) malloc(Width1*Height1*sizeof(float));
    N=(float*) malloc(Width2*Height2*sizeof(float));
    Pout=(float*) malloc(Width2*Height1*sizeof(float));

    intialize_matrix(M,Height1,Width1);
    intialize_matrix(N,Height2,Width2);
    //cout<<"M"<<endl;
   // Print_matrix(M,Height1,Width1);
   // cout<<endl;
   // cout<<"N"<<endl;
    //Print_matrix(N,Height2,Width2);

    //cout<<endl;
     clock_t start = clock();
    MatrixMulOnDevice(M,N,Pout);

    //I added code
    cudaDeviceSynchronize();
        cudaError_t err1 = cudaDeviceSynchronize();
   if (err1!= cudaSuccess) {
      printf("%s in %s at line %d\n",cudaGetErrorString(err1), __FILE__, __LINE__);
        exit(EXIT_FAILURE);
        }
    clock_t stop = clock();
    double time_spent = (double)(stop - start) / CLOCKS_PER_SEC;
    cout<<"Time taken to complete the entire wrapper computation is: "<<time_spent<<" seconds.\n";

   // cout<<"P"<<endl;
 //Print_matrix(Pout,Height1,Width2);
    //2.28394e+10 2.28395e+10
    //5.32934e+12 5.32934e+12
  

 cout<<Pout[0]<<" "<<Pout[1999];
    return 0;
}

Time taken to complete only the Kernel computation is: 0.446976 seconds.
Time taken to complete the entire wrapper computation is: 0.759228 seconds.
5.32934e+12 5.33333e+12


###Speedup for 1000x500 * 500x1000 matrices
Speedup = Time in Sequential / Time in Parallel

speedup of kernel only = 1.77262/0.053848 = 33x times speedup

speedup of whole wrapper function = 1.77262/0.283998= 6x times speedup

###Speedup for 1000x1000 matrices
Speedup = Time in Sequential / Time in Parallel

speedup of kernel only = 3.54652/0.105868 = 34x times speedup

speedup of whole wrapper function = 3.54652/0.332278 = 11x times speedup